In [ ]:

# Global channel Effect
mu_global = pm.LogNormal("mu_global", mu = np.log([10000, 20000, 30000]), sigma = np.log([1.2, 1.3, 1.6]), dims = "channel") # (3,)


# Market Specific Channel Deviation
mar_dev = pm.HalfNormal("mar_dev", sigma = [10, 20, 30], dims = "channel") # (3,)
z_dev_mar = pm.Normal("z_dev_mar", mu = 0, sigma = 1, dims = ("market", "channel")) # (3, 3)

# Product Specific Channel Deviation
prod_dev = pm.HalfNormal("prod_dev", sigma = [5, 4, 3], dims = ("channel")) # (3,)
z_dev_prod = pm.Normal("z_dev_mar", mu = 0, sigma = 1, dims = ("product", "channel")) # (3,3)

beta_mpc = pm.Deterministic("beta_mpc", 
                            mu_global[None, None, :] + 
                            mar_dev[None, None, :] * z_dev_mar[:, None, :] +
                            prod_dev[None, None, :] * z_dev_prod[None, :, :], 
                            dims = ("market", "product", "channel"))

beta_obs = beta_mpc[market_idx, product_idx, :] # (N,)

media_weekly_contri = beta_obs * X_media_spend # (N, C)

weekly_channel_contribution = pm.deterministic("media_weekly_contri", 
                                               media_weekly_contri, 
                                               dims = ("obs", "channel"))

weekly_channel_contribution = weekly_channel_contribution.stack(sample = ("chain", "draw")) # Create dim samples - chain * draw                                               
mean_weekly_contrib = weekly_channel_contribution.mean(axis = 1) # Mean of channel contribution over time

ROI_distribution = weekly_channel_contribution / X_media_spend


